## CompanyIndexScraper Sheet
---

A sheet dedicated to the analysis and testing of scraper company indices. 

Together with an example of use.

Scraper acquires data in 6 segments:
- `COMPANY` (indexes in relation to company),
- `COMPANY_QQ` (indexes in relation to company Quarterly - Quarterly),
- `COMPANY_YY` (indexes in relation to company Year - Year),
- `SECTOR` (indexes in relation to sector),
- `SECTOR_QQ` (indexes in relation to sector Quarterly - Quarterly),
- `SECTOR_YY` (indexes in relation to sector Year - Year)

In [1]:
from scripts import CompanyIndexScraper
cis = CompanyIndexScraper()
# Scrap data - example scrap CCC (company ticker) 
cis.scrap_data("CCC")  

In [2]:
cis.get_report_index(CompanyIndexScraper.IndexType.COMPANY)

,Cena / Wartość księgowa,Cena / Wartość księgowa Grahama,Cena / Przychody ze sprzedaży,Cena / Zysk,Cena / Zysk operacyjny,EV / Przychody ze sprzedaży,EV / EBIT,EV / EBITDA,ROE,ROA,...,Wskaźnik źródeł finansowania inwestycji,Zadłużenie ogólne,Zadłużenie kapitału własnego,Zastosowanie kapitału obcego,Wskaźnik ogólnej sytuacji finansowej,I stopień pokrycia,Płynność gotówkowa,Płynność szybka,Płynność bieżąca,Pokrycie zobowiązań należnościami
2004/Q4,2.92,4.60,1.32,17.68,12.35,1.29,12.05,11.16,16.49,11.42,...,13.53,0.31,0.44,0.41,1.13,2.73,0.67,1.56,3.01,0.89
2005/Q4,7.10,10.91,3.62,27.33,21.84,3.52,21.24,19.83,25.98,21.44,...,88.54,0.18,0.21,0.25,0.70,2.86,0.94,1.69,4.16,0.75
2006/Q1,8.44,13.86,4.32,34.15,27.91,4.38,28.29,26.10,24.72,19.59,...,529.02,0.21,0.26,0.30,0.77,2.56,0.06,1.11,3.38,1.04
2006/Q2,10.16,17.21,4.44,30.02,25.34,4.49,25.65,23.89,33.84,24.58,...,104.07,0.27,0.38,0.39,0.95,2.44,0.10,0.87,2.62,0.76
2006/Q3,10.55,18.39,4.67,32.62,28.17,4.76,28.75,26.63,32.34,22.11,...,93.30,0.32,0.46,0.45,1.05,2.35,0.09,0.87,2.27,0.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023/Q1,5.05,NaN,0.26,NaN,NaN,0.65,-97.78,11.40,-71.86,-4.76,...,130.26,0.91,13.77,1.79,0.24,0.13,0.09,0.20,0.89,0.10
2023/Q2,4.48,NaN,0.38,NaN,NaN,0.74,-502.37,12.06,-34.04,-3.74,...,350.22,0.87,7.93,1.76,0.38,0.22,0.10,0.21,0.89,0.11
2023/Q3,3.42,NaN,0.29,NaN,71.38,0.65,159.56,9.67,-25.10,-2.56,...,78.74,0.88,8.66,1.66,0.36,0.22,0.12,0.22,0.85,0.10
2023/Q4,4.78,NaN,0.43,NaN,21.84,0.80,41.20,9.73,-6.67,-0.76,...,169.04,0.87,7.60,1.77,0.40,0.22,0.06,0.16,0.81,0.09


## Scrap Index Data 

In [3]:
from scripts import DatesScraper
from scripts import CompanyIndexScraper
import pandas as pd

cis = CompanyIndexScraper()
ds  = DatesScraper() 
COMPANY_TICKER_LIST = pd.read_csv("../database/designData/WIGs.csv")["WIG_ODZIEZ"].dropna().to_numpy()

In [4]:
dates_maper = ds.get_date_map("CCC")
cis.scrap_data("CCC")
report = cis.get_report_index(CompanyIndexScraper.IndexType.COMPANY)
report = dates_maper.join(report, how="inner")
report.reset_index(drop=True, inplace=True)

In [5]:
reports = {
    CompanyIndexScraper.IndexType.COMPANY   : pd.DataFrame(),
    CompanyIndexScraper.IndexType.SECTOR    : pd.DataFrame(),
    CompanyIndexScraper.IndexType.COMPANY_YY: pd.DataFrame(),
    CompanyIndexScraper.IndexType.SECTOR_YY : pd.DataFrame(),
    CompanyIndexScraper.IndexType.COMPANY_QQ: pd.DataFrame(),
    CompanyIndexScraper.IndexType.SECTOR_QQ : pd.DataFrame(),
}

for ticker in COMPANY_TICKER_LIST:
    dates_maper = ds.get_date_map(ticker)
    cis.scrap_data(ticker)
    for key in reports:
        index = cis.get_report_index(key)
        index = dates_maper.join(index, how="inner")
        index.reset_index(drop=True, inplace=True)
        index.insert(1, "Ticker", ticker)
        [key] = pd.concat([reports[key], index],
                                 ignore_index=True)


CCC
CDL
EAH
HRP
IPO
LBW
LPP
MIR
MON
PRT
SFG
SNW
WTN
